In [ ]:
!pip install -q transformers bitsandbytes accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, BitsAndBytesConfig


In [ ]:
model_id = "nacielo/whispherMusic"

# Quantization config for 4-bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load model with quantization
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load processor
processor = AutoProcessor.from_pretrained(model_id)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/2.23k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/339 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.72k [00:00<?, ?B/s]

In [ ]:
!pip install -q datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [ ]:
from datasets import load_dataset
import torch

# Load a test audio sample
ds = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
sample = ds[0]["audio"]

# Preprocess input and convert to float16
inputs = processor(sample["array"], sampling_rate=sample["sampling_rate"], return_tensors="pt").to("cuda")
inputs["input_features"] = inputs["input_features"].half()  # ⚠️ Convert to float16

# Generate predictions
with torch.no_grad():
    predicted_ids = model.generate(inputs["input_features"])

# Decode output
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)
print("Transcription:", transcription[0])


Transcription: this is a tutorial with a male voice talking in a deeper voice and playing a rhythm on a soft steel pan. the recording is noisy and the recording is roomy.


In [4]:
# Install requirements (uncomment if needed)
# !pip install torchaudio transformers librosa psutil

import torch
import torchaudio
import time
import psutil
import os
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Upload or define file
try:
    from google.colab import files
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
except:
    file_name = "your_audio_file.wav"  # Replace with your local file

# Load model and processor
model_id = "openai/whisper-tiny"  # You can change to base/small/medium
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)

# Move to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Load and preprocess audio
waveform, sr = torchaudio.load(file_name)
target_sr = processor.feature_extractor.sampling_rate
if sr != target_sr:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
    waveform = resampler(waveform)

# Measure preprocessing time
start_pre = time.time()
inputs = processor(waveform.squeeze().numpy(), sampling_rate=target_sr, return_tensors="pt").to(device)
end_pre = time.time()

# Measure peak memory before inference
process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss / 1024**2  # in MB

# Measure inference time (latency)
start_infer = time.time()
with torch.no_grad():
    predicted_ids = model.generate(inputs["input_features"])
end_infer = time.time()

# Measure memory after inference
mem_after = process.memory_info().rss / 1024**2  # in MB

# Decode output
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# Calculate metrics
latency = end_infer - start_infer
preprocess_time = end_pre - start_pre
total_time = end_infer - start_pre
memory_used = mem_after - mem_before
throughput = 1 / latency if latency > 0 else 0  # samples/sec

# Print results
print("🔊 Transcription:")
print(transcription)
print("\n📊 Performance Metrics:")
print(f"Preprocessing Time     : {preprocess_time:.3f} seconds")
print(f"Inference Latency      : {latency:.3f} seconds")
print(f"Total Time (End-to-End): {total_time:.3f} seconds")
print(f"Memory Usage Increase  : {memory_used:.2f} MB")
print(f"Throughput             : {throughput:.2f} audio/sec")


Saving whispher.opus to whispher (2).opus
🔊 Transcription:
 I Hello, are you I This is Abhishek doing chat from product. I need a sample for this I Can face model. God of I might

📊 Performance Metrics:
Preprocessing Time     : 0.017 seconds
Inference Latency      : 2.798 seconds
Total Time (End-to-End): 2.815 seconds
Memory Usage Increase  : 79.44 MB
Throughput             : 0.36 audio/sec


In [3]:
# Install requirements (uncomment if needed)
# !pip install torchaudio transformers librosa psutil

import torch
import torchaudio
import time
import psutil
import os
from transformers import WhisperProcessor, WhisperForConditionalGeneration

# Upload or define file
try:
    from google.colab import files
    uploaded = files.upload()
    file_name = list(uploaded.keys())[0]
except:
    file_name = "your_audio_file.wav"  # Replace with your local file

# Load model and processor
model_id = "openai/whisper-tiny"  # You can change to base/small/medium
processor = WhisperProcessor.from_pretrained(model_id)
model = WhisperForConditionalGeneration.from_pretrained(model_id)

# Apply dynamic quantization to optimize model for CPU inference
model = torch.quantization.quantize_dynamic(
    model,  # The model to quantize
    {torch.nn.Linear},  # Apply to all linear layers (this is typical for most transformer models)
    dtype=torch.qint8  # Quantizing to 8-bit integer
)

# Move to CPU (since we are quantizing for CPU inference)
device = "cpu"
model.to(device)

# Load and preprocess audio
waveform, sr = torchaudio.load(file_name)
target_sr = processor.feature_extractor.sampling_rate
if sr != target_sr:
    resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=target_sr)
    waveform = resampler(waveform)

# Measure preprocessing time
start_pre = time.time()
inputs = processor(waveform.squeeze().numpy(), sampling_rate=target_sr, return_tensors="pt").to(device)
end_pre = time.time()

# Measure peak memory before inference
process = psutil.Process(os.getpid())
mem_before = process.memory_info().rss / 1024**2  # in MB

# Measure inference time (latency)
start_infer = time.time()
with torch.no_grad():
    predicted_ids = model.generate(inputs["input_features"])
end_infer = time.time()

# Measure memory after inference
mem_after = process.memory_info().rss / 1024**2  # in MB

# Decode output
transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)[0]

# Calculate metrics
latency = end_infer - start_infer
preprocess_time = end_pre - start_pre
total_time = end_infer - start_pre
memory_used = mem_after - mem_before
throughput = 1 / latency if latency > 0 else 0  # samples/sec

# Print results
print("🔊 Transcription:")
print(transcription)
print("\n📊 Performance Metrics:")
print(f"Preprocessing Time     : {preprocess_time:.3f} seconds")
print(f"Inference Latency      : {latency:.3f} seconds")
print(f"Total Time (End-to-End): {total_time:.3f} seconds")
print(f"Memory Usage Increase  : {memory_used:.2f} MB")
print(f"Throughput             : {throughput:.2f} audio/sec")


Saving whispher.opus to whispher (1).opus
🔊 Transcription:
 I think it's more than that.

📊 Performance Metrics:
Preprocessing Time     : 0.019 seconds
Inference Latency      : 1.781 seconds
Total Time (End-to-End): 1.800 seconds
Memory Usage Increase  : 0.26 MB
Throughput             : 0.56 audio/sec
